In [ ]:
# HelmetGuard – Synthetic Data Generation

#This notebook generates synthetic construction site images (safe / unsafe) using a diffusion model.
#Images are saved into `data_synth/safe` and `data_synth/unsafe`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/helmetguard"
SYNTH_DIR = os.path.join(BASE_DIR, "data_synth")

os.makedirs(os.path.join(SYNTH_DIR, "safe"), exist_ok=True)
os.makedirs(os.path.join(SYNTH_DIR, "unsafe"), exist_ok=True)

print(SYNTH_DIR)


/content/drive/MyDrive/helmetguard/data_synth


In [ ]:
!pip install -q diffusers transformers accelerate safetensors


In [ ]:
import torch
from diffusers import StableDiffusionPipeline


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [ ]:
!pip install -q diffusers transformers accelerate safetensors huggingface_hub

import torch
from diffusers import StableDiffusionPipeline
from huggingface_hub import login

login()



In [ ]:
!pip install -q diffusers transformers accelerate safetensors huggingface_hub

import torch
from diffusers import StableDiffusionPipeline


In [ ]:
model_id = "Manojb/stable-diffusion-2-1-base"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to("cuda")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
PROMPTS_SAFE = [
    "high resolution photo of a construction site, many construction workers, all wearing yellow safety helmets and reflective vests, realistic, daytime",
    "photo of a road construction site, workers with safety helmets and vests, realistic, sunny day",
    "wide angle photo of a building construction site, several workers, all wearing safety helmets, ultra realistic"
]

PROMPTS_UNSAFE = [
    "high resolution photo of a construction site, several construction workers, ONE worker without any helmet, bare head, no helmet on the head, realistic, daytime",
    "photo of a construction site, a worker without safety helmet standing near other workers, bare head, no hard hat, realistic, sunny",
    "wide angle photo of a building construction site, some workers wearing helmets and one worker with NO helmet, bare head, ultra realistic"
]

NEG_SAFE = "cartoon, illustration, painting, 3d render, low quality, blurry, distorted faces"
NEG_UNSAFE = "cartoon, illustration, painting, 3d render, low quality, blurry, distorted faces, safety helmet, hard hat"


In [ ]:
from PIL import Image
import itertools
import os

BASE_DIR = "/content/drive/MyDrive/helmetguard"
SYNTH_DIR = os.path.join(BASE_DIR, "data_synth")

def generate_images_for_class(prompts, label, num_images=50, negative_prompt=""):
    save_dir = os.path.join(SYNTH_DIR, label)
    os.makedirs(save_dir, exist_ok=True)


    existing = [f for f in os.listdir(save_dir) if f.endswith(".png")]
    start_idx = len(existing)

    prompt_cycle = itertools.cycle(prompts)

    for i in range(num_images):
        idx = start_idx + i
        prompt = next(prompt_cycle)
        print(f"{label} {idx}: {prompt}")

        image = pipe(
            prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=30,
            guidance_scale=7.5
        ).images[0]

        image_path = os.path.join(save_dir, f"{label}_{idx:05d}.png")
        image.save(image_path)


In [ ]:
generate_images_for_class(PROMPTS_SAFE, "safe", num_images=200, negative_prompt=NEG_SAFE)
generate_images_for_class(PROMPTS_UNSAFE, "unsafe", num_images=200, negative_prompt=NEG_UNSAFE)


safe 310: high resolution photo of a construction site, many construction workers, all wearing yellow safety helmets and reflective vests, realistic, daytime


  0%|          | 0/30 [00:00<?, ?it/s]

safe 311: photo of a road construction site, workers with safety helmets and vests, realistic, sunny day


  0%|          | 0/30 [00:00<?, ?it/s]

safe 312: wide angle photo of a building construction site, several workers, all wearing safety helmets, ultra realistic


...

  0%|          | 0/30 [00:00<?, ?it/s]

unsafe 508: high resolution photo of a construction site, several construction workers, ONE worker without any helmet, bare head, no helmet on the head, realistic, daytime


  0%|          | 0/30 [00:00<?, ?it/s]

unsafe 509: photo of a construction site, a worker without safety helmet standing near other workers, bare head, no hard hat, realistic, sunny


  0%|          | 0/30 [00:00<?, ?it/s]